# 🤗 Transformers Episodio 4 - Parte 4


In [1]:
import utils
import importlib
importlib.reload(utils)

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from utils import get_submission, submit, get_tokenizer_and_model,\
                  compute_metrics, load_dfs, tokenize, get_train_args


import transformers
from transformers import Trainer
transformers.logging.set_verbosity_warning()

def read_scores():
    s = !kaggle competitions submissions nlp-getting-started 
    df = pd.DataFrame([(l.split()[0], l.split()[-2]) for l in s[2:]], columns=["Archivo", "Score"]).set_index("Archivo")
    display(df)
    return df

utils.nb_set_width()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [2]:
model_id = "episodio_4_pred_4"
pred_file_name = f"{model_id}.csv"
submit_message = "Episodio 4 Prediccion 4 Bert Varias Epochs"

model_name = "bert-base-cased"
tokenizer, model = get_tokenizer_and_model(model_name)

df_base, df_test = load_dfs()
df_train, df_val = train_test_split(df_base, test_size=0.066)
ds_train, ds_val, ds_test = tokenize(tokenizer, df_train, df_val, df_test)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [4]:
trainer = Trainer(model=model, args=get_train_args(model_id, epochs=4, save_steps=500), 
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.508900,0.466396,0.809145,0.769231,0.808081,0.733945
2,0.388600,0.465995,0.817097,0.779904,0.815000,0.747706
3,0.314500,0.651032,0.817097,0.771144,0.842391,0.711009
4,0.225300,0.765736,0.817097,0.781991,0.808824,0.756881


TrainOutput(global_step=3556, training_loss=0.3580994697201775, metrics={'train_runtime': 2844.4639, 'train_samples_per_second': 9.998, 'train_steps_per_second': 1.25, 'total_flos': 9462951556300800.0, 'train_loss': 0.3580994697201775, 'epoch': 4.0})

In [5]:
df_res = get_submission(trainer, ds_test, pred_file_name)

In [6]:
submit(pred_file_name, submit_message)

stdout=
Successfully submitted to Natural Language Processing with Disaster Tweets
stderr=
100%|██████████| 22.2k/22.2k [00:00<00:00, 120kB/s]



In [7]:
# Leaderbord - 744
read_scores();

,Score
Archivo,
episodio_4_pred_4.csv,0.81979
episodio_4_pred_2_checkpoint_2000.csv,0.80968
episodio_4_pred_2.csv,0.81734
episodio_4_pred_1.csv,0.81274
bert_first_submission.csv,0.80049


In [ ]:
!sudo shutdown now